In [ ]:
import transformers, torch
from collections import Counter
import torch, re, gc, os, sys
import numpy as np
import pandas as pd

p="/kaggle/input/google-code-golf-2025/code_golf_utils"
sys.path.append(p)
from code_golf_utils import *

DEVICE = torch.device('cuda')
os.environ['OMP_NUM_THREADS'] = '2'
os.environ['TOKENIZERS_PARALLELISM'] = 'true'
MODEL_PATH = '/kaggle/input/qwen2.5-coder/transformers/3b-instruct/1'
transformers.set_seed(2025)
tokenizer = transformers.Qwen2Tokenizer.from_pretrained(MODEL_PATH)
gpipe = transformers.AutoModelForCausalLM.from_pretrained(MODEL_PATH, device_map="auto", torch_dtype = torch.float16,)
torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)

In [ ]:
def getDSR(usec, aonly=True):
    global gpipe
    input_ids = tokenizer(usec, return_tensors="pt").to("cuda")
    output = gpipe.generate(**input_ids, max_new_tokens=800, do_sample=True, temperature=0.7, top_k=20, top_p=0.8, repetition_penalty=1.2)
    res = tokenizer.decode(output[0], skip_special_tokens=True)[len(usec):]
    return res

In [ ]:
#https://www.kaggle.com/code/davidbnn92/task-tagging/notebook
ARCTaskClasses = [['007bbfb7', ['image_repetition', 'fractal_repetition']],
['00d62c1b', ['loop_filling']],
['017c7c7b', ['recoloring', 'pattern_expansion', 'pattern_repetition', 'image_expansion']],
['025d127b', ['pattern_modification']],
['045e512c', ['pattern_expansion', 'direction_guessing']],
['0520fde7', ['detect_wall', 'separate_images', 'pattern_intersection']],
['05269061', ['image_filling', 'pattern_expansion', 'diagonals']],
['05f2a901', ['pattern_moving', 'direction_guessing', 'bring_patterns_close']],
['06df4c85', ['detect_grid', 'connect_the_dots', 'grid_coloring']],
['08ed6ac7', ['measure_length', 'order_numbers', 'associate_colors_to_ranks', 'recoloring']],
['09629e4f', ['detect_grid', 'separate_images', 'count_tiles', 'take_minimum', 'enlarge_image', 'create_grid', 'adapt_image_to_grid']],
['0962bcdd', ['pattern_expansion']],
['0a938d79', ['direction_guessing', 'draw_line_from_point', 'pattern_expansion']],
['0b148d64', ['detect_grid', 'separate_images', 'find_the_intruder', 'crop']],
['0ca9ddb6', ['pattern_expansion', 'associate_patterns_to_colors']],
['0d3d703e', ['associate_colors_to_colors']],
['0dfd9992', ['image_filling', 'pattern_expansion']],
['0e206a2e', ['associate_patterns_to_patterns', 'pattern_repetition', 'pattern_rotation', 'pattern_reflection', 'pattern_juxtaposition']],
['10fcaaa3', ['pattern_expansion', 'image_repetition']],
['11852cab', ['pattern_expansion']],
['1190e5a7', ['detect_grid', 'count_hor_lines', 'count_ver_lines', 'detect_background_color', 'color_guessing', 'create_image_from_info']],
['137eaa0f', ['pattern_juxtaposition']],
['150deff5', ['pattern_coloring', 'pattern_deconstruction', 'associate_colors_to_patterns']],
['178fcbfb', ['direction_guessing', 'draw_line_from_point']],
['1a07d186', ['bring_patterns_close', 'find_the_intruder']],
['1b2d62fb', ['detect_wall', 'separate_images', 'pattern_intersection']],
['1b60fb0c', ['pattern_deconstruction', 'pattern_rotation', 'pattern_expansion']],
['1bfc4729', ['pattern_expansion']],
['1c786137', ['detect_enclosure', 'crop']],
['1caeab9d', ['pattern_moving', 'pattern_alignment']],
['1cf80156', ['crop']],
['1e0a9b12', ['pattern_moving', 'gravity']],
['1e32b0e9', ['detect_grid', 'separate_images', 'image_repetition', 'pattern_completion']],
['1f0c79e5', ['pattern_expansion', 'diagonals', 'direction_guessing']],
['1f642eb9', ['image_within_image', 'projection_unto_rectangle']],
['1f85a75f', ['crop', 'find_the_intruder']],
['1f876c06', ['connect_the_dots', 'diagonals']],
['1fad071e', ['count_patterns', 'associate_images_to_numbers']],
['2013d3e2', ['pattern_deconstruction', 'crop']],
['2204b7a8', ['proximity_guessing', 'recoloring']],
['22168020', ['pattern_expansion']],
['22233c11', ['pattern_expansion', 'size_guessing']],
['2281f1f4', ['direction_guessing', 'draw_line_from_point', 'pattern_intersection']],
['228f6490', ['pattern_moving', 'loop_filling', 'shape_guessing', 'x_marks_the_spot']],
['22eb0ac0', ['connect_the_dots', 'color_matching']],
['234bbc79', ['recoloring', 'bring_patterns_close', 'crop']],
['23581191', ['draw_line_from_point', 'pattern_intersection']],
['239be575', ['detect_connectedness', 'associate_images_to_bools']],
['23b5c85d', ['measure_area', 'take_minimum', 'crop']],
['253bf280', ['connect_the_dots', 'direction_guessing']],
['25d487eb', ['draw_line_from_point', 'direction_guessing', 'color_guessing']],
['25d8a9c8', ['detect_hor_lines', 'recoloring', 'remove_noise']],
['25ff71a9', ['pattern_moving']],
['264363fd', ['pattern_repetition', 'pattern_juxtaposition', 'draw_line_from_point']],
['272f95fa', ['detect_grid', 'mimic_pattern', 'grid_coloring']],
['27a28665', ['associate_colors_to_patterns', 'take_negative', 'associate_images_to_patterns']],
['28bf18c6', ['crop', 'pattern_repetition']],
['28e73c20', ['ex_nihilo', 'mimic_pattern']],
['29623171', ['detect_grid', 'separate_images', 'count_tiles', 'take_maximum', 'grid_coloring']],
['29c11459', ['draw_line_from_point', 'count_tiles']],
['29ec7d0e', ['image_filling', 'pattern_expansion', 'detect_grid', 'pattern_repetition']],
['2bcee788', ['pattern_reflection', 'direction_guessing', 'image_filling', 'background_filling']],
['2bee17df', ['draw_line_from_border', 'count_tiles', 'take_maximum']],
['2c608aff', ['draw_line_from_point', 'projection_unto_rectangle']],
['2dc579da', ['detect_grid', 'find_the_intruder', 'crop']],
['2dd70a9a', ['draw_line_from_point', 'direction_guessing', 'maze']],
['2dee498d', ['detect_repetition', 'crop', 'divide_by_n']],
['31aa019c', ['find_the_intruder', 'remove_noise', 'contouring']],
['321b1fc6', ['pattern_repetition', 'pattern_juxtaposition']],
['32597951', ['find_the_intruder', 'recoloring']],
['3345333e', ['pattern_completion', 'pattern_reflection', 'remove_noise']],
['3428a4f5', ['detect_wall', 'separate_images', 'pattern_differences']],
['3618c87e', ['gravity']],
['3631a71a', ['image_filling', 'pattern_expansion', 'pattern_rotation']],
['363442ee', ['detect_wall', 'pattern_repetition', 'pattern_juxtaposition']],
['36d67576', ['pattern_repetition', 'pattern_juxtaposition', 'pattern_reflection', 'pattern_rotation']],
['36fdfd69', ['recoloring', 'rectangle_guessing']],
['3906de3d', ['gravity']],
['39a8645d', ['count_patterns', 'take_maximum', 'crop']],
['39e1d7f9', ['detect_grid', 'pattern_repetition', 'grid_coloring']],
['3aa6fb7a', ['pattern_completion', 'pattern_rotation']],
['3ac3eb23', ['draw_pattern_from_point', 'pattern_repetition']],
['3af2c5a8', ['image_repetition', 'image_reflection', 'image_rotation']],
['3bd67248', ['draw_line_from_border', 'diagonals','pattern_repetition']],
['3bdb4ada', ['recoloring','pattern_repetition', 'holes']],
['3befdf3e', ['take_negative', 'pattern_expansion']],
['3c9b0459', ['image_rotation']],
['3de23699', ['take_negative', 'crop', 'rectangle_guessing']],
['3e980e27', ['pattern_repetition', 'pattern_juxtaposition', 'direction_guessing', 'pattern_reflection']],
['3eda0437', ['rectangle_guessing', 'recoloring', 'measure_area', 'take_maximum']],
['3f7978a0', ['crop', 'rectangle_guessing', 'find_the_intruder']],
['40853293', ['connect_the_dots']],
['4093f84a', ['gravity', 'recoloring', 'projection_unto_rectangle']],
['41e4d17e', ['draw_line_from_point', 'pattern_repetition']],
['4258a5f9', ['pattern_repetition', 'contouring']],
['4290ef0e', ['pattern_moving', 'concentric', 'crop']],
['42a50994', ['remove_noise', 'count_tiles']],
['4347f46a', ['loop_filling', 'color_guessing']],
['444801d8', ['pattern_repetition', 'pattern_expansion', 'rectangle_guessing']],
['445eab21', ['measure_area', 'take_maximum']],
['447fd412', ['pattern_repetition', 'draw_pattern_from_point', 'pattern_resizing']],
['44d8ac46', ['loop_filling', 'rectangle_guessing']],
['44f52bb0', ['detect_symmetry', 'associate_images_to_bools']],
['4522001f', ['image_rotation', 'pairwise_analogy']],
['4612dd53', ['pattern_completion', 'rectangle_guessing']],
['46442a0e', ['image_repetition', 'image_reflection']],
['469497ad', ['image_resizing', 'draw_line_from_point', 'diagonals']],
['46f33fce', ['pattern_resizing', 'image_resizing']],
['47c1f68c', ['detect_grid', 'find_the_intruder', 'crop', 'recolor', 'color_guessing', 'image_repetition', 'image_reflection']],
['484b58aa', ['image_filling', 'pattern_expansion', 'pattern_repetition']],
['48d8fb45', ['find_the_intruder', 'crop']],
['4938f0c2', ['pattern_expansion', 'pattern_rotation', 'pattern_reflection']],
['496994bd', ['pattern_reflection']],
['49d1d64f', ['pattern_expansion', 'image_expansion']],
['4be741c5', ['summarize']],
['4c4377d9', ['image_repetition', 'image_reflection']],
['4c5c2cf0', ['pattern_expansion', 'pattern_rotation', 'pattern_reflection']],
['50846271', ['pattern_completion', 'recoloring']],
['508bd3b6', ['draw_line_from_point', 'direction_guessing', 'pattern_reflection']],
['50cb2852', ['holes', 'rectangle_guessing']],
['5117e062', ['find_the_intruder', 'crop', 'recoloring']],
['5168d44c', ['direction_guessing', 'recoloring', 'contouring', 'pattern_moving']],
['539a4f51', ['pattern_expansion', 'image_expansion']],
['53b68214', ['pattern_expansion', 'image_expansion']],
['543a7ed5', ['contouring', 'loop_filling']],
['54d82841', ['pattern_expansion', 'gravity']],
['54d9e175', ['detect_grid', 'separate_images', 'associate_images_to_images']],
['5521c0d9', ['pattern_moving', 'measure_length']],
['5582e5ca', ['count_tiles', 'dominant_color']],
['5614dbcf', ['remove_noise', 'image_resizing']],
['56dc2b01', ['gravity', 'direction_guessing', 'pattern_expansion']],
['56ff96f3', ['pattern_completion', 'rectangle_guessing']],
['57aa92db', ['draw_pattern_from_point', 'pattern_repetition', 'pattern_resizing']],
['5ad4f10b', ['color_guessing', 'remove_noise', 'recoloring', 'crop', 'image_resizing']],
['5bd6f4ac', ['rectangle_guessing', 'crop']],
['5c0a986e', ['draw_line_from_point', 'diagonals', 'direction_guessing']],
['5c2c9af4', ['rectangle_guessing', 'pattern_expansion']],
['5daaa586', ['detect_grid', 'crop', 'draw_line_from_point', 'direction_guessing']],
['60b61512', ['pattern_completion']],
['6150a2bd', ['image_rotation']],
['623ea044', ['draw_line_from_point', 'diagonals']],
['62c24649', ['image_repetition', 'image_reflection', 'image_rotation']],
['63613498', ['recoloring', 'compare_image', 'detect_wall']],
['6430c8c4', ['detect_wall', 'separate_images', 'take_complement', 'pattern_intersection']],
['6455b5f5', ['measure_area', 'take_maximum', 'take_minimum', 'loop_filling', 'associate_colors_to_ranks']],
['662c240a', ['separate_images', 'detect_symmetry',  'find_the_intruder', 'crop']],
['67385a82', ['recoloring', 'measure_area', 'associate_colors_to_bools']],
['673ef223', ['recoloring', 'draw_line_from_point', 'portals']],
['6773b310', ['detect_grid', 'separate_images', 'count_tiles', 'associate_colors_to_numbers']],
['67a3c6ac', ['image_reflection']],
['67a423a3', ['pattern_intersection', 'contouring']],
['67e8384a', ['image_repetition', 'image_reflection', 'image_rotation']],
['681b3aeb', ['pattern_moving', 'jigsaw', 'crop', 'bring_patterns_close']],
['6855a6e4', ['pattern_moving', 'direction_guessing', 'x_marks_the_spot']],
['68b16354', ['image_reflection']],
['694f12f3', ['rectangle_guessing', 'loop_filling', 'measure_area', 'associate_colors_to_ranks']],
['6a1e5592', ['pattern_moving', 'jigsaw', 'recoloring']],
['6aa20dc0', ['pattern_repetition', 'pattern_juxtaposition', 'pattern_resizing']],
['6b9890af', ['pattern_moving', 'pattern_resizing', 'crop', 'x_marks_the_spot']],
['6c434453', ['replace_pattern']],
['6cdd2623', ['connect_the_dots', 'find_the_intruder', 'remove_noise']],
['6cf79266', ['rectangle_guessing', 'recoloring']],
['6d0160f0', ['detect_grid', 'separate_image', 'find_the_intruder', 'pattern_moving']],
['6d0aefbc', ['image_repetition', 'image_reflection']],
['6d58a25d', ['draw_line_from_point']],
['6d75e8bb', ['rectangle_guessing', 'pattern_completion']],
['6e02f1e3', ['count_different_colors', 'associate_images_to_numbers']],
['6e19193c', ['draw_line_from_point', 'direction_guessing', 'diagonals']],
['6e82a1ae', ['recoloring', 'count_tiles', 'associate_colors_to_numbers']],
['6ecd11f4', ['color_palette', 'recoloring', 'pattern_resizing', 'crop']],
['6f8cd79b', ['ex_nihilo', 'contouring']],
['6fa7a44f', ['image_repetition', 'image_reflection']],
['72322fa7', ['pattern_repetition', 'pattern_juxtaposition']],
['72ca375d', ['find_the_intruder', 'detect_symmetry', 'crop']],
['73251a56', ['image_filling', 'diagonal_symmetry']],
['7447852a', ['pattern_expansion', 'pairwise_analogy']],
['7468f01a', ['crop', 'image_reflection']],
['746b3537', ['crop', 'direction_guessing']],
['74dd1130', ['image_reflection', 'diagonal_symmetry']],
['75b8110e', ['separate_images', 'image_juxtaposition']],
['760b3cac', ['pattern_reflection', 'direction_guessing']],
['776ffc46', ['recoloring', 'associate_colors_to_patterns', 'detect_enclosure', 'find_the_intruder']],
['77fdfe62', ['recoloring', 'color_guessing', 'detect_grid', 'crop']],
['780d0b14', ['detect_grid', 'summarize']],
['7837ac64', ['detect_grid', 'color_guessing', 'grid_coloring', 'crop', 'extrapolate_image_from_grid']],
['794b24be', ['count_tiles', 'associate_images_to_numbers']],
['7b6016b9', ['loop_filling', 'background_filling', 'color_guessing']],
['7b7f7511', ['separate_images', 'detect_repetition', 'crop']],
['7c008303', ['color_palette', 'detect_grid', 'recoloring', 'color_guessing', 'separate_images', 'crop']],
['7ddcd7ec', ['draw_line_from_point', 'direction_guessing', 'diagonals']],
['7df24a62', ['pattern_repetition', 'pattern_rotation', 'pattern_juxtaposition', 'out_of_boundary']],
['7e0986d6', ['color_guessing', 'remove_noise']],
['7f4411dc', ['rectangle_guessing', 'remove_noise']],
['7fe24cdd', ['image_repetition', 'image_rotation']],
['80af3007', ['crop', 'pattern_resizing', 'image_resizing', 'fractal_repetition']],
['810b9b61', ['recoloring', 'detect_closed_curves']],
['82819916', ['pattern_repetition', 'color_guessing', 'draw_line_from_point', 'associate_colors_to_colors']],
['83302e8f', ['detect_grid', 'detect_closed_curves', 'rectangle_guessing', 'associate_colors_to_bools', 'loop_filling']],
['834ec97d', ['draw_line_from_border', 'pattern_repetition', 'spacing', 'measure_distance_from_side']],
['8403a5d5', ['draw_line_from_point', 'pattern_repetition', 'direction_guessing']],
['846bdb03', ['pattern_moving', 'pattern_reflection', 'crop', 'color_matching', 'x_marks_the_spot']],
['855e0971', ['draw_line_from_point', 'direction_guessing', 'separate_images', 'holes']],
['85c4e7cd', ['color_guessing', 'recoloring', 'color_permutation']],
['868de0fa', ['loop_filling', 'color_guessing', 'measure_area', 'even_or_odd', 'associate_colors_to_bools']],
['8731374e', ['rectangle_guessing', 'crop', 'draw_line_from_point']],
['88a10436', ['pattern_repetition', 'pattern_juxtaposition']],
['88a62173', ['detect_grid', 'separate_images', 'find_the_intruder', 'crop']],
['890034e9', ['pattern_repetition', 'rectangle_guessing', 'contouring']],
['8a004b2b', ['pattern_repetition', 'pattern_resizing', 'pattern_juxtaposition', 'rectangle_guessing', 'crop']],
['8be77c9e', ['image_repetition', 'image_reflection']],
['8d5021e8', ['image_repetition', 'image_reflection']],
['8d510a79', ['draw_line_from_point', 'detect_wall', 'direction_guessing', 'associate_colors_to_bools']],
['8e1813be', ['recoloring', 'color_guessing', 'direction_guesing' 'crop', 'image_within_image']],
['8e5a5113', ['detect_wall', 'separate_images', 'image_repetition', 'image_rotation']],
['8eb1be9a', ['pattern_repetition', 'image_filling']],
['8efcae92', ['separate_images', 'rectangle_guessing', 'count_tiles', 'take_maximum', 'crop']],
['8f2ea7aa', ['crop', 'fractal_repetition']],
['90c28cc7', ['crop', 'rectangle_guessing', 'summarize']],
['90f3ed37', ['pattern_repetition', 'recoloring']],
['913fb3ed', ['contouring', 'associate_colors_to_colors']],
['91413438', ['count_tiles', 'algebra', 'image_repetition']],
['91714a58', ['find_the_intruder', 'remove_noise']],
['9172f3a0', ['image_resizing']],
['928ad970', ['rectangle_guessing', 'color_guessing', 'draw_rectangle']],
['93b581b8', ['pattern_expansion', 'color_guessing', 'out_of_boundary']],
['941d9a10', ['detect_grid', 'loop_filling', 'pairwise_analogy']],
['94f9d214', ['separate_images', 'take_complement', 'pattern_intersection']],
['952a094c', ['rectangle_guessing', 'inside_out']],
['9565186b', ['separate_shapes', 'count_tiles', 'recoloring', 'take_maximum', 'associate_color_to_bools']],
['95990924', ['pattern_expansion']],
['963e52fc', ['image_expansion', 'pattern_expansion']],
['97999447', ['draw_line_from_point', 'pattern_expansion']],
['97a05b5b', ['pattern_moving', 'pattern_juxtaposition', 'crop', 'shape_guessing']],
['98cf29f8', ['pattern_moving', 'bring_patterns_close']],
['995c5fa3', ['take_complement', 'detect_wall', 'separate_images', 'associate_colors_to_images', 'summarize']],
['99b1bc43', ['take_complement', 'detect_wall', 'separate_images', 'pattern_intersection']],
['99fa7670', ['draw_line_from_point', 'pattern_expansion']],
['9aec4887', ['pattern_moving', 'x_marks_the_spot', 'crop', 'recoloring', 'color_guessing']],
['9af7a82c', ['separate_images', 'count_tiles', 'summarize', 'order_numbers']],
['9d9215db', ['pattern_expansion', 'pattern_reflection', 'pattern_rotation']],
['9dfd6313', ['image_reflection', 'diagonal_symmetry']],
['9ecd008a', ['image_filling', 'pattern_expansion', 'pattern_reflection', 'pattern_rotation', 'crop']],
['9edfc990', ['background_filling', 'holes']],
['9f236235', ['detect_grid', 'summarize', 'image_reflection']],
['a1570a43', ['pattern_moving', 'rectangle_guessing', 'x_marks_the_spot']],
['a2fd1cf0', ['connect_the_dots']],
['a3325580', ['separate_shapes', 'count_tiles', 'take_maximum', 'summarize', 'remove_intruders']],
['a3df8b1e', ['pattern_expansion', 'draw_line_from_point', 'diagonals', 'bounce']],
['a416b8f3', ['image_repetition']],
['a48eeaf7', ['pattern_moving', 'bring_patterns_close', 'gravity', 'direction_guessing']],
['a5313dff', ['loop_filling']],
['a5f85a15', ['recoloring', 'pattern_modification', 'pairwise_analogy']],
['a61ba2ce', ['pattern_moving', 'bring_patterns_close', 'crop', 'jigsaw']],
['a61f2674', ['separate_shapes', 'count_tiles', 'take_maximum', 'take_minimum', 'recoloring', 'associate_colors_to_ranks', 'remove_intruders']],
['a64e4611', ['background_filling', 'rectangle_guessing']],
['a65b410d', ['pattern_expansion', 'count_tiles', 'associate_colors_to_ranks']],
['a68b268e', ['detect_grid', 'separate_images', 'pattern_juxtaposition']],
['a699fb00', ['pattern_expansion', 'connect_the_dots']],
['a740d043', ['crop', 'detect_background_color', 'recoloring']],
['a78176bb', ['draw_parallel_line', 'direction_guessing', 'remove_intruders']],
['a79310a0', ['pattern_moving', 'recoloring', 'pairwise_analogy']],
['a85d4709', ['separate_images', 'associate_colors_to_images', 'summarize']],
['a87f7484', ['separate_images', 'find_the_intruder', 'crop']],
['a8c38be5', ['pattern_moving', 'jigsaw', 'crop']],
['a8d7556c', ['recoloring', 'rectangle_guessing']],
['a9f96cdd', ['replace_pattern', 'out_of_boundary']],
['aabf363d', ['recoloring', 'color_guessing', 'remove_intruders']],
['aba27056', ['pattern_expansion', 'draw_line_from_point', 'diagonals']],
['ac0a08a4', ['image_resizing', 'count_tiles', 'size_guessing']],
['ae3edfdc', ['bring_patterns_close', 'gravity']],
['ae4f1146', ['separate_images', 'count_tiles', 'crop']],
['aedd82e4', ['recoloring', 'separate_shapes', 'count_tiles', 'take_minimum', 'associate_colors_to_bools']],
['af902bf9', ['ex_nihilo', 'x_marks_the_spot']],
['b0c4d837', ['measure_length', 'associate_images_to_numbers']],
['b190f7f5', ['separate_images', 'image_expasion', 'color_palette', 'image_resizing', 'replace_pattern']],
['b1948b0a', ['recoloring', 'associate_colors_to_colors']],
['b230c067', ['recoloring', 'separate_shapes', 'find_the_intruder', 'associate_colors_to_bools']],
['b27ca6d3', ['find_the_intruder', 'count_tiles', 'contouring']],
['b2862040', ['recoloring', 'detect_closed_curves', 'associate_colors_to_bools']],
['b527c5c6', ['pattern_expansion', 'draw_line_from_point', 'contouring', 'direction_guessing', 'size_guessing']],
['b548a754', ['pattern_expansion', 'pattern_modification', 'x_marks_the_spot']],
['b60334d2', ['replace_pattern']],
['b6afb2da', ['recoloring', 'replace_pattern', 'rectangle_guessing']],
['b7249182', ['pattern_expansion']],
['b775ac94', ['pattern_expansion', 'pattern_repetition', 'recoloring', 'pattern_rotation', 'pattern_reflection', 'direction_guessing', 'pattern_juxtaposition']],
['b782dc8a', ['pattern_expansion', 'maze']],
['b8825c91', ['pattern_completion', 'pattern_rotation', 'pattern_reflection']],
['b8cdaf2b', ['pattern_expansion', 'draw_line_from_point', 'diagonals', 'pairwise_analogy']],
['b91ae062', ['image_resizing', 'size_guessing', 'count_different_colors']],
['b94a9452', ['crop', 'take_negative']],
['b9b7f026', ['find_the_intruder', 'summarize']],
['ba26e723', ['pattern_modification', 'pairwise_analogy', 'recoloring']],
['ba97ae07', ['pattern_modification', 'pairwise_analogy', 'rettangle_guessing', 'recoloring']],
['bb43febb', ['loop_filling', 'rettangle_guessing']],
['bbc9ae5d', ['pattern_expansion', 'image_expansion']],
['bc1d5164', ['pattern_moving', 'pattern_juxtaposition', 'crop', 'pairwise_analogy']],
['bd4472b8', ['detect_wall', 'pattern_expansion', 'ex_nihilo', 'color_guessing', 'color_palette']],
['bda2d7a6', ['recoloring', 'pairwise_analogy', 'pattern_modification', 'color_permutation']],
['bdad9b1f', ['draw_line_from_point', 'direction_guessing', 'recoloring', 'take_intersection']],
['be94b721', ['separate_shapes', 'count_tiles', 'take_maximum', 'crop']],
['beb8660c', ['pattern_moving', 'count_tiles', 'order_numbers']],
['c0f76784', ['loop_filling', 'measure_area', 'associate_colors_to_numbers']],
['c1d99e64', ['draw_line_from_border', 'detect_grid']],
['c3e719e8', ['image_repetition', 'image_expansion', 'count_different_colors', 'take_maximum']],
['c3f564a4', ['pattern_expansion', 'image_filling']],
['c444b776', ['detect_grid', 'separate_images', 'find_the_intruder', 'image_repetition']],
['c59eb873', ['image_resizing']],
['c8cbb738', ['pattern_moving', 'jigsaw', 'crop']],
['c8f0f002', ['recoloring', 'associate_colors_to_colors']],
['c909285e', ['find_the_intruder', 'crop', 'rectangle_guessing']],
['c9e6f938', ['image_repetition', 'image_reflection']],
['c9f8e694', ['recoloring', 'pattern_repetition', 'color_palette']],
['caa06a1f', ['pattern_expansion', 'image_filling']],
['cbded52d', ['detect_grid', 'separate_images', 'pattern_modification', 'pattern_repetition', 'pattern_juxtaposition', 'connect_the_dots']],
['cce03e0d', ['image_repetition', 'image_expansion', 'pairwise_analogy']],
['cdecee7f', ['summarize', 'pairwise_analogy']],
['ce22a75a', ['replace_pattern']],
['ce4f8723', ['detect_wall', 'separate_images', 'take_complement', 'take_intersection']],
['ce602527', ['crop', 'size_guessing', 'shape_guessing', 'find_the_intruder', 'remove_intruder']],
['ce9e57f2', ['recoloring', 'count_tiles', 'take_half']],
['cf98881b', ['detect_wall', 'separate_images', 'pattern_juxtaposition']],
['d037b0a7', ['pattern_expansion', 'draw_line_from_point']],
['d06dbe63', ['pattern_expansion', 'pairwise_analogy']],
['d07ae81c', ['draw_line_from_point', 'diagonals', 'color_guessing']],
['d0f5fe59', ['separate_shapes', 'count_shapes', 'associate_images_to_numbers', 'pairwise_analogy']],
['d10ecb37', ['crop']],
['d13f3404', ['image_expansion', 'draw_line_from_point', 'diagonals']],
['d22278a0', ['pattern_expansion', 'pairwise_analogy']],
['d23f8c26', ['crop', 'image_expansion']],
['d2abd087', ['separate_shapes', 'count_tiles', 'associate_colors_to_numbers', 'recoloring']],
['d364b489', ['pattern_expansion']],
['d406998b', ['recoloring', 'one_yes_one_no', 'cylindrical']],
['d43fd935', ['draw_line_from_point', 'direction_guessing', 'projection_unto_rectangle']],
['d4469b4b', ['dominant_color', 'associate_images_to_colors']],
['d4a91cb9', ['connect_the_dots', 'direction_guessing']],
['d4f3cd78', ['rectangle_guessing', 'recoloring', 'draw_line_from_point']],
['d511f180', ['associate_colors_to_colors']],
['d5d6de2d', ['loop_filling', 'replace_pattern', 'remove_intruders']],
['d631b094', ['count_tiles', 'dominant_color', 'summarize']],
['d687bc17', ['bring_patterns_close', 'gravity', 'direction_guessing', 'find_the_intruder', 'remove_intruders']],
['d6ad076f', ['bridges', 'connect_the_dots', 'draw_line_from_point']],
['d89b689b', ['pattern_juxtaposition', 'summarize', 'direction_guessing']],
['d8c310e9', ['pattern_expansion', 'pattern_repetition', 'pattern_completion']],
['d90796e8', ['replace_pattern']],
['d9f24cd1', ['draw_line_from_point', 'gravity', 'obstacles']],
['d9fac9be', ['find_the_intruder', 'summarize', 'x_marks_the_spot']],
['dae9d2b5', ['pattern_juxtaposition', 'separate_images', 'recoloring']],
['db3e9e38', ['pattern_expansion', 'out_of_boundary']],
['db93a21d', ['contouring', 'draw_line_from_point', 'measure_area', 'measure_length', 'algebra']],
['dbc1a6ce', ['connect_the_dots']],
['dc0a314f', ['pattern_completion', 'crop']],
['dc1df850', ['contouring', 'pattern_expansion', 'out_of_boundary']],
['dc433765', ['pattern_moving', 'direction_guessing', 'only_one']],
['ddf7fa4f', ['color_palette', 'recoloring']],
['de1cd16c', ['separate_images', 'count_tiles', 'take_maximum', 'summarize']],
['ded97339', ['connect_the_dots']],
['e179c5f4', ['pattern_expansion', 'bouncing']],
['e21d9049', ['pattern_expansion', 'draw_line_from_point', 'color_palette']],
['e26a3af2', ['remove_noise', 'separate_images']],
['e3497940', ['detect_wall', 'separate_images', 'image_reflection', 'image_juxtaposition']],
['e40b9e2f', ['pattern_expansion', 'pattern_reflection', 'pattern_rotation']],
['e48d4e1a', ['count_tiles', 'pattern_moving', 'detect_grid', 'out_of_boundary']],
['e5062a87', ['pattern_repetition', 'pattern_juxtaposition']],
['e509e548', ['recoloring', 'associate_colors_to_shapes', 'homeomorphism']],
['e50d258f', ['separate_images', 'detect_background_color', 'crop', 'count_tiles', 'take_maximum']],
['e6721834', ['pattern_moving', 'pattern_juxtaposition', 'crop']],
['e73095fd', ['loop_filling', 'rectangle_guessing']],
['e76a88a6', ['pattern_repetition', 'pattern_juxtaposition']],
['e8593010', ['holes', 'count_tiles', 'loop_filling', 'associate_colors_to_numbers']],
['e8dc4411', ['pattern_expansion', 'direction_guessing']],
['e9614598', ['pattern_expansion', 'direction_guessing', 'measure_length']],
['e98196ab', ['detect_wall', 'separate_images', 'image_juxtaposition']],
['e9afcf9a', ['pattern_modification']],
['ea32f347', ['separate_shapes', 'count_tiles', 'recoloring', 'associate_colors_to_ranks']],
['ea786f4a', ['pattern_modification', 'draw_line_from_point', 'diagonals']],
['eb281b96', ['image_repetition', 'image_reflection']],
['eb5a1d5d', ['summarize']],
['ec883f72', ['pattern_expansion', 'draw_line_from_point', 'diagonals']],
['ecdecbb3', ['pattern_modification', 'draw_line_from_point']],
['ed36ccf7', ['image_rotation']],
['ef135b50', ['draw_line_from_point', 'bridges', 'connect_the_dots']],
['f15e1fac', ['draw_line_from_point', 'gravity', 'obstacles', 'direction_guessing']],
['f1cefba8', ['draw_line_from_point', 'pattern_modification']],
['f25fbde4', ['crop', 'image_resizing']],
['f25ffba3', ['pattern_repetition', 'pattern_reflection']],
['f2829549', ['detect_wall', 'separate_images', 'take_complement', 'pattern_intersection']],
['f35d900a', ['pattern_expansion']],
['f5b8619d', ['pattern_expansion', 'draw_line_from_point', 'image_repetition']],
['f76d97a5', ['take_negative', 'recoloring', 'associate_colors_to_colors']],
['f8a8fe49', ['pattern_moving', 'pattern_reflection']],
['f8b3ba0a', ['detect_grid', 'find_the_intruder', 'dominant_color', 'count_tiles', 'summarize', 'order_numbers']],
['f8c80d96', ['pattern_expansion', 'background_filling']],
['f8ff0b80', ['separate_shapes', 'count_tiles', 'summarize', 'order_numbers']],
['f9012d9b', ['pattern_expansion', 'pattern_completion', 'crop']],
['fafffa47', ['separate_images', 'take_complement', 'pattern_intersection']],
['fcb5c309', ['rectangle_guessing', 'separate_images', 'count_tiles', 'take_maximum', 'crop', 'recoloring']],
['fcc82909', ['pattern_expansion', 'separate_images', 'count_different_colors']],
['feca6190', ['pattern_expansion', 'image_expansion', 'draw_line_from_point', 'diagonals']],
['ff28f65a', ['count_shapes', 'associate_images_to_numbers']],
['ff805c23', ['pattern_expansion', 'pattern_completion', 'crop']]]

In [ ]:
#https://www.kaggle.com/code/seshurajup/golf-tasks-2-arc-agi/notebook?scriptVersionId=253630815
dfall = pd.read_csv('/kaggle/input/golf-task-2-arc-agi/task2agi.csv')
ARCTaskClasses = {t[0]:t[1] for t in ARCTaskClasses}
dfall['index'] = dfall['agi_id'].map(lambda x: x.replace('.json',''))
dfall['Tagging'] = dfall['index'].map(ARCTaskClasses)

In [ ]:
#https://www.kaggle.com/code/taylorsamarel/qwen2-5-32b-arc-local-score-32-solved-script
import zipfile, json, os, copy

def check(solution, task_num, valall=False):
    task_data = load_examples(task_num)
    #print(task_num, max(1, 2500 - len(solution.encode('utf-8'))))
    try:
        namespace = {}
        exec(solution, namespace)
        if 'p' not in namespace: return False
        all_examples = task_data['train'] + task_data['test'] + task_data['arc-gen']
        examples_to_check = all_examples if valall else all_examples[:3]
        for example in examples_to_check:
            input_grid = copy.deepcopy(example['input'])
            expected = example['output']
            try:
                actual = namespace['p'](input_grid)
                if actual != expected:
                    return False
            except:
                return False
        return True
    except Exception as e:
        return False

#Add break for infinite loops - runtime break

In [ ]:
#Harder no public script solutions yet - mostly object copy, motion, gravity and pattern fills
r=[5,9,17,18,19,20,22,23,25,27,29,30,34,35,36,42,44,46,54,
   61,62,64,66,69,71,74,76,77,79,80,86,89,90,93,96,99,
   101,102,105,107,109,110,111,112,114,117,118,119,122,123,124,
   125,133,134,137,138,143,148,153,154,156,157,158,159,
   160,161,165,168,170,173,174,175,182,185,189,191,192,195,197,198,
   202,203,204,205,206,208,209,212,214,216,218,219,221,222,224,225,
   233,234,237,238,240,242,243,244,245,247,250,255,257,
   260,264,265,268,270,271,273,277,279,280,281,284,285,286,287,288,291,295,
   306,308,314,319,324,328,330,333,338,340,341,343,349,351,355,358,359,
   362,364,366,367,370,377,379,382,383,387,390,392,394,398,400]

In [ ]:
def cleanQwenCode(s):
    p = r'```python\s*(.*?)\s*```'
    s = re.findall(p, s, re.DOTALL)[0]
    return s

In [ ]:
#Prompt from https://www.kaggle.com/code/katsuyanomura/simpletasksolved
#Please write Python code as concisely as possible. Function names should be p(g), and all variables should be one character long. Indentation should be one half-width space. If possible, use lambda expressions (p=lambda g:) to write concisely. If range appears three or more times, apply R=range as an alias.

for task_num in r[:3]:
    examples = load_examples(task_num)
    info=dfall[dfall['task_no']==task_num]['agi_id'].values[0]
    info2=dfall[dfall['task_no']==task_num]['Tagging'].values[0]
    g = examples['train'][0]['input']
    prompt= "Write a python function for the ARC-AGI challenge to solve task id " + str(info) + " using def p(g) and code golf style python code. All variables should be one character long. Indentation should be one half-width space. If possible, use lambda expressions (p=lambda g:) to write concisely. If range appears three or more times, apply R=range as an alias. Input and output will be a 2d array. Should end with return g and no print outputs or examples."  
    #print(prompt)
    res=getDSR(prompt)
    try:
        solution=cleanQwenCode(res)
        print(task_num, info, '*'*40)
        print(solution)
        val=check(solution, task_num, valall=True)
        print(val, '='*40)
        if val:
            f=open('task' + str(task_num).zfill(3) + '.py','w')
            f.write(solution)
            f.close()
    except:pass
    #break

# Ｈ𝐀𝑷𝑷𝓎 🇰𝗮𝘨𝘨🇱𝖎Ｎɢ 💯